In [1]:
# load modules
import pandas as pd

In [2]:
# read in a list of urls from cms data source: 
# https://www.cms.gov/icd10m/version39.0-fullcode-cms/fullcode_cms/P0378.html
cms_urls = pd.read_csv("cms_mdc_urls.csv")

In [3]:
# scrape data from each url
df = pd.DataFrame()
for url in cms_urls['cms_url'].unique():
    # list all tables from the url
    tables_list = pd.read_html(url)
    # the second table contains the reference table
    df_table = tables_list[2]
    # concat 74 tables into 1 df
    df = pd.concat([df, df_table])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29598 entries, 0 to 264
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DX          29576 non-null  object 
 1   MDC         29576 non-null  object 
 2   MS-DRG      29576 non-null  object 
 3   Unnamed: 3  0 non-null      float64
 4   DX.1        29292 non-null  object 
 5   MDC.1       29292 non-null  object 
 6   MS-DRG.1    29292 non-null  object 
 7   Unnamed: 7  0 non-null      float64
 8   DX.2        29295 non-null  object 
 9   MDC.2       29295 non-null  object 
 10  MS-DRG.2    29295 non-null  object 
dtypes: float64(2), object(9)
memory usage: 2.7+ MB


In [5]:
# select all object columns (dropping empty float64 columns)
df = df.select_dtypes(include='object')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29598 entries, 0 to 264
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   DX        29576 non-null  object
 1   MDC       29576 non-null  object
 2   MS-DRG    29576 non-null  object
 3   DX.1      29292 non-null  object
 4   MDC.1     29292 non-null  object
 5   MS-DRG.1  29292 non-null  object
 6   DX.2      29295 non-null  object
 7   MDC.2     29295 non-null  object
 8   MS-DRG.2  29295 non-null  object
dtypes: object(9)
memory usage: 2.3+ MB


In [6]:
# reshape to stack overflow columns
cols0 = df[['DX', 'MDC', 'MS-DRG']]
cols1 = df[['DX.1', 'MDC.1', 'MS-DRG.1']]
cols1 = cols1.rename(columns={'DX.1': 'DX', 'MDC.1': 'MDC', 'MS-DRG.1': 'MS-DRG'})
cols2 = df[['DX.2', 'MDC.2', 'MS-DRG.2']]
cols2 = cols2.rename(columns={'DX.2': 'DX', 'MDC.2': 'MDC', 'MS-DRG.2': 'MS-DRG'})
df = pd.concat([cols0, cols1, cols2])

In [7]:
# drop any duplicate rows
df = df.drop_duplicates()
# drop any empty rows
df = df.dropna()
# reorder for legibility
df = df[['DX', 'MS-DRG', 'MDC']]
# check final result
df.head()